In [ ]:
import pandas as pd
import numpy as np
import os as os
import openpyxl as op
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# PREPARACIÓN E IMPORTACIÓN DE DATOS DIAN (RECAUDO BRUTO)

# Ruta base de la carpeta "datasets" (AJUSTAR ESTA RUTA SI ES NECESARIO)
ruta_datasets = r"C:\Users\Kayak\OneDrive\Escritorio\U\2025-2\IA_aplicada\proyecto\datasets"
# Nombre del primer archivo que es tipo xlsx
Recaudo_Bruto = "Estadisticas-de-Recaudo-bruto-por-seccionales-y-tipo-de-impuesto-2005-2025.xlsx"

# creamos ruta para importar el primer archivo
ruta_Recaudo_Bruto = os.path.join(ruta_datasets, Recaudo_Bruto)

# carga del archivo usando la ruta, saltándonos las primeras 5 filas y las últimas 10
df_impuestos = pd.read_excel(ruta_Recaudo_Bruto,  skiprows=5,skipfooter=10)
# al ver el dataframe, notamos que la fila 1 y 2 están fusionadas por lo que debemos eliminar una
df_impuestos=df_impuestos.drop(index=0)

# de otro lado la columna AÑO debe ser rellenada (forward fill)
df_impuestos['AÑO'] = df_impuestos['AÑO'].ffill()

# Ahora bien, vamos a manejar datos solamente de 2018 a 2023, por lo que filtramos el dataframe
df_impuestos['AÑO'] = pd.to_numeric(df_impuestos['AÑO'], errors='coerce')  # Convertir a numérico
df_impuestos = df_impuestos[(df_impuestos['AÑO'] >= 2018) & (df_impuestos['AÑO'] <= 2023)]
df_impuestos['AÑO'] = df_impuestos['AÑO'].astype(int)
df_impuestos.reset_index(inplace=True, drop=True)

# Normalizar los nombres en la columna DIRECCIÓN_SECCIONAL
df_impuestos["DIRECCIÓN SECCIONAL"] = (
    df_impuestos["DIRECCIÓN SECCIONAL"]
    .str.strip()  # elimina espacios antes/después
    .str.upper() ) # pasa todo a mayúsculas

## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# CONVERSIÓN DE UNIDADES (Millones a Miles de $COP)


# hay columnas tipo texto asi que vamos a ignorarlas para trabajar sin errores en una conversión de formato.
columnas_excluidas = ['AÑO', 'DIRECCIÓN SECCIONAL']
columnas_a_convertir = df_impuestos.columns.drop(columnas_excluidas)

# conversión solo a esas columnas. Reemplazamos no-números con NaN y luego multiplicamos.
df_impuestos[columnas_a_convertir] = df_impuestos[columnas_a_convertir].apply(pd.to_numeric, errors='coerce')
# Multiplicación para pasar de Millones a Miles (Millones * 1000)
df_impuestos[columnas_a_convertir] = df_impuestos[columnas_a_convertir] * 1000
# convertimos los valores NaN a 0
df_impuestos = df_impuestos.fillna(0)


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# MAPPING DE SECCIONALES A DEPARTAMENTO Y CÓDIGO (DIAN

# Diccionario ciudad -> departamento (ajustado a mayúsculas)
ciudades_departamentos = {
    "ADUANAS BARRANQUILLA": "ATLÁNTICO",
    "ADUANAS BOGOTA": "CUNDINAMARCA",
    "ADUANAS CALI": "VALLE DEL CAUCA",
    "ADUANAS CARTAGENA": "BOLÍVAR",
    "ADUANAS DE CUCUTA": "NORTE DE SANTANDER",
    "ADUANAS DE MEDELLIN": "ANTIOQUIA",
    "IMPUESTOS BOGOTA": "CUNDINAMARCA",
    "GRANDES CONTRIBUYENTES BOGOTA": "BOGOTÁ, D.C.",
    "OPERERATIVA DE GRANDES CONTRIBUYENTES": "BOGOTÁ, D.C.",
    "OPERATIVA GRANDES CONTRIBUYENTES": "BOGOTÁ, D.C.",

    "ARAUCA": "ARAUCA",
    "ARMENIA": "QUINDÍO",
    "BARRANCABERMEJA": "SANTANDER",
    "BARRANQUILLA": "ATLÁNTICO",
    "BUCARAMANGA": "SANTANDER",
    "BUENAVENTURA": "VALLE DEL CAUCA",
    "CALI": "VALLE DEL CAUCA",
    "CARTAGENA": "BOLÍVAR",
    "CUCUTA": "NORTE DE SANTANDER",
    "FLORENCIA": "CAQUETÁ",
    "GIRARDOT": "CUNDINAMARCA",
    "IBAGUE": "TOLIMA",
    "INIRIDA": "GUAINÍA",
    "IPIALES": "NARIÑO",
    "LETICIA": "AMAZONAS",
    "MAICAO": "LA GUAJIRA",
    "MANIZALES": "CALDAS",
    "MEDELLIN": "ANTIOQUIA",
    "MITU": "VAUPÉS",
    "MONTERIA": "CÓRDOBA",
    "NEIVA": "HUILA",
    "PALMIRA": "VALLE DEL CAUCA",
    "PASTO": "NARIÑO",
    "PEREIRA": "RISARALDA",
    "POPAYAN": "CAUCA",
    "PUERTO ASIS": "PUTUMAYO",
    "PUERTO CARREÑO": "VICHADA",
    "QUIBDO": "CHOCÓ",
    "RIOHACHA": "LA GUAJIRA",
    "SAN ANDRES": "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA",
    "SAN JOSE DEL GUAVIARE": "GUAVIARE",
    "SANTA MARTA": "MAGDALENA",
    "SINCELEJO": "SUCRE",
    "SOGAMOSO": "BOYACÁ",
    "TULUA": "VALLE DEL CAUCA",
    "TUNJA": "BOYACÁ",
    "URABA": "ANTIOQUIA",
    "VALLEDUPAR": "CESAR",
    "VILLAVICENCIO": "META",
    "YOPAL": "CASANARE"
}

# Diccionario departamento -> código
departamento_codigos = {
    "ANTIOQUIA": 5,
    "ATLÁNTICO": 8,
    "BOGOTÁ, D.C.": 11,
    "BOLÍVAR": 13,
    "BOYACÁ": 15,
    "CALDAS": 17,
    "CAQUETÁ": 18,
    "CAUCA": 19,
    "CESAR": 20,
    "CÓRDOBA": 23,
    "CUNDINAMARCA": 25,
    "CHOCÓ": 27,
    "HUILA": 41,
    "LA GUAJIRA": 44,
    "MAGDALENA": 47,
    "META": 50,
    "NARIÑO": 52,
    "NORTE DE SANTANDER": 54,
    "QUINDÍO": 63,
    "RISARALDA": 66,
    "SANTANDER": 68,
    "SUCRE": 70,
    "TOLIMA": 73,
    "VALLE DEL CAUCA": 76,
    "ARAUCA": 81,
    "CASANARE": 85,
    "PUTUMAYO": 86,
    "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA": 88,
    "AMAZONAS": 91,
    "GUAINÍA": 94,
    "GUAVIARE": 95,
    "VAUPÉS": 97,
    "VICHADA": 99
}

# eliminación de filas de totales por año
df_impuestos_arreglado = df_impuestos[~df_impuestos["DIRECCIÓN SECCIONAL"].str.contains("TOTAL AÑO", case=False, na=False)].copy()
# Asignación de departamento
df_impuestos_arreglado["departamento"] = df_impuestos_arreglado["DIRECCIÓN SECCIONAL"].map(ciudades_departamentos)

# Asignación de código de departamento
df_impuestos_arreglado["dpto"] = df_impuestos_arreglado["departamento"].map(departamento_codigos)
df_impuestos_arreglado['dpto'] = df_impuestos_arreglado['dpto'].apply(pd.to_numeric, errors='coerce')
df_impuestos_arreglado = df_impuestos_arreglado.dropna(subset=["dpto"])
df_impuestos_arreglado["dpto"] = df_impuestos_arreglado["dpto"].astype(int)

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# CONSOLIDACIÓN DIAN POR DPTO Y AÑO 

# Agrupar por 'dpto' y 'AÑO' y sumar los valores de recaudo para consolidar
# todas las seccionales (Aduanas, Impuestos, Grandes Contribuyentes) en un
# único total por cada departamento y año.
df_DIAN_consolidado = df_impuestos_arreglado.groupby(['dpto', 'AÑO']).sum(numeric_only=True).reset_index()

# Función para limpiar los nombres de columnas (eliminar espacios no deseados y \xa0)
def clean_col_name(col):
    col = col.replace('\xa0', ' ') # Reemplazar espacio no-breaking con espacio normal
    col = col.strip()             # Eliminar espacios iniciales/finales (incluyendo \n)
    col = ' '.join(col.split())   # Colapsar múltiples espacios internos a uno solo
    return col

# Aplicar la limpieza a los nombres de las columnas del DataFrame de la DIAN
df_DIAN_consolidado.columns = [clean_col_name(col) for col in df_DIAN_consolidado.columns]

# Opcional: limpiar columnas que no son de impuestos después de la suma
columnas_a_mantener_dian = ['dpto', 'AÑO'] + [col for col in df_DIAN_consolidado.columns if col not in ['dpto', 'AÑO', 'DIRECCIÓN SECCIONAL', 'departamento']]
df_DIAN_consolidado = df_DIAN_consolidado[columnas_a_mantener_dian]


# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# IMPORTACIÓN Y PROCESAMIENTO DE DATOS EAM (DANE)

#dado que son varios datasets, vamos a hacer un bucle para procesarlos todos de forma
#eficiente. Algo importante es que en la agrupación de la base de datos no queremos
# tomar en cuenta las columnas que no son de impuestos, por lo que haremos un renombramiento
# y una selección previa de las columnas que nos interesan, tampoco vamos a eliminarlas pues
# las necesitaremos para hacer merge de las bases más adelante.

# indicación de la ruta de los datasets y el separador de los archivos csv
ruta_datasets = r'C:\Users\Kayak\OneDrive\Escritorio\U\2025-2\IA_aplicada\proyecto\datasets'
CSV_SEPARATOR = ';'

# Diccionario de configuración de archivos y formato exacto por año
archivos = {
    2018: {'nombre': 'EAM_ANONIMIZADA_2018.csv', 'formato': 'csv'},
    2019: {'nombre': 'EAM_ANONIMIZADA_2019.csv', 'formato': 'csv'},
    2020: {'nombre': 'EAM_ANONIMIZADA_2020.xlsx', 'formato': 'xlsx'},
    2021: {'nombre': 'EAM_ANONIMIZADA_2021.xlsx', 'formato': 'xlsx'},
    2022: {'nombre': 'EAM_ANONIMIZADA_2022.xlsx', 'formato': 'xlsx'},
    2023: {'nombre': 'EAM_ANONIMIZADA_2023.xlsx', 'formato': 'xlsx'}
}

# Diccionario de decodificación y selección de columnas
columnas_renombre = {
    'dpto': 'DEPARTAMENTO',
    'periodo': 'AÑO',
    'C3R20C3': 'Impuesto de Renta para la Equidad',
    'c3r25c3': 'Impuesto del 4 x 1 000',
    'c3r37c3': 'Impuesto de Industria y Comercio',
    'c3r38c3': 'Impuesto Predial y sobre Vehículos'
}
columnas_seleccion = list(columnas_renombre.keys()) # Las columnas originales que buscamos
columnas_impuestos = list(columnas_renombre.values())[2:] # Solo los nombres de los impuestos finales

# Lista para almacenar los DataFrames procesados anualmente
list_df_anuales = []

# Bucle de Procesamiento
for año, dataset in archivos.items():
    nombre_archivo = dataset['nombre']
    formato_archivo = dataset['formato']
    ruta_archivo = os.path.join(ruta_datasets, nombre_archivo)
    df_EAM = None

    # Importación Condicional
    try:
        if formato_archivo == 'xlsx':
            df_EAM = pd.read_excel(ruta_archivo)
        elif formato_archivo == 'csv':
            df_EAM = pd.read_csv(ruta_archivo, sep=CSV_SEPARATOR)
        else:
            print(f" Error: Formato '{formato_archivo}' no reconocido.")
            continue

        # Estandarización de nombres: convierte los nombres de columnas a minúsculas
        df_EAM.columns = df_EAM.columns.str.lower().str.strip()
        print(f"✅ Importado y nombres estandarizados: {nombre_archivo}")

    except FileNotFoundError:
        print(f" Error: Archivo NO encontrado en la ruta para el año {año}.")
        continue
    except Exception as e:
        print(f" Error al leer el archivo {nombre_archivo}. Detalles: {e}")
        continue

    #Selección y Renombramiento
    columnas_a_seleccionar_limpias = [col.lower() for col in columnas_seleccion]

    try:
        # Aquí aplicamos la selección de columnas.
        df_EAM_util = df_EAM[columnas_a_seleccionar_limpias].copy()
    except KeyError as e:
        print(f" ERROR CRÍTICO en el año {año}: Columnas no encontradas. Faltan: {e}. Revisar el archivo.")
        print(f"Columnas disponibles: {df_EAM.columns.tolist()}")
        continue

    # Renombramos usando los nombres limpios como clave
    columnas_renombre_limpio = {k.lower(): v for k, v in columnas_renombre.items()}
    df_EAM_util = df_EAM_util.rename(columns=columnas_renombre_limpio)


    #Convertimos  los datos a numero (Necesario para sumar después de leer CSV/TXT)
    for col in columnas_impuestos:
        df_EAM_util[col] = pd.to_numeric(df_EAM_util[col], errors='coerce')

    # Agrupación y Suma por Departamento
    df_EAM_sumado_por_dpto = df_EAM_util.groupby('DEPARTAMENTO').sum(numeric_only=True).reset_index()

    # hay una columna que se llama año pero en la base de la DIAN se llama AÑO por
    # lo que vamos a crear una nueva columna llamada AÑO y le asignamos el valor del año actual
    df_EAM_sumado_por_dpto['AÑO'] = año

    #Almacenamiento para concatenación
    list_df_anuales.append(df_EAM_sumado_por_dpto)


# --- Consolidación Final EAM ---
# Concatenación vertical (apilamiento) de todos los resúmenes anuales.
df_EAM_consolidado = pd.concat(list_df_anuales, ignore_index=True)
df_EAM_consolidado.rename(columns={'DEPARTAMENTO': 'dpto'}, inplace=True)
print("\n--- PROCESO TERMINADO ---")
print("\nPrimeras filas de la EAM):")
print(df_EAM_consolidado.head(10))
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# CRUCE FINAL DE DATOS 

# Usamos df_DIAN_consolidado (el agrupado) y la doble clave ['dpto', 'AÑO']
df_EAM_impuestos = pd.merge(df_EAM_consolidado, df_DIAN_consolidado, on=['dpto', 'AÑO'], how='outer')
df_EAM_impuestos = df_EAM_impuestos.fillna(0)

# Eliminación de columnas vacías.
# con clean_col_name() vamos a limpiar los nombres de las columnas que queremos eliminar.
#luego las eliminamos del dataframe final.
#manteniendo el resto de columnas de EAM y DIAN.

columnas_a_eliminar_sucias = [
    'PRODUCTOS PLÁSTICOS 10/', 'BEBIDAS ULTRA PROCESADAS 11/',
    'PRODUCTOS COMESTIBLES 12/', 'IMPUESTO ESPECIAL PARA EL CATATUMBO14/']

columnas_a_eliminar = [clean_col_name(col) for col in columnas_a_eliminar_sucias]
df_EAM_impuestos = df_EAM_impuestos.drop(columns=columnas_a_eliminar, errors='raise')
df_EAM_impuestos.to_excel("EAM final.xlsx", index=False)

C:\Users\Kayak\AppData\Local\Temp\ipykernel_8464\1973093605.py:231: DtypeWarning: Columns (302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363) have mixed types. Specify dtype option on import or set low_memory=False.
  df_EAM = pd.read_csv(ruta_archivo, sep=CSV_SEPARATOR)


✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2018.csv


C:\Users\Kayak\AppData\Local\Temp\ipykernel_8464\1973093605.py:231: DtypeWarning: Columns (302) have mixed types. Specify dtype option on import or set low_memory=False.
  df_EAM = pd.read_csv(ruta_archivo, sep=CSV_SEPARATOR)


✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2019.csv
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2020.xlsx
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2021.xlsx
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2022.xlsx
✅ Importado y nombres estandarizados: EAM_ANONIMIZADA_2023.xlsx

--- PROCESO TERMINADO ---
DataFrames consolidados para los años: 2018 a 2023
Total de registros consolidados (Filas): 138

Primeras filas del DataFrame Consolidado (EAM):
   DEPARTAMENTO   AÑO  Impuesto de Renta para la Equidad  \
0             5  2018                                0.0   
1             8  2018                                0.0   
2            11  2018                                0.0   
3            13  2018                                0.0   
4            15  2018                                0.0   
5            17  2018                                0.0   
6            19  2018                                0.0   
7            20  2018        

In [28]:
df_EAM_impuestos = df_EAM_impuestos.fillna(0)
df_EAM_impuestos.to_excel("EAM final.xlsx", index=False)

In [ ]:
import pandas as pd
import numpy as np
import os as os
import openpyxl as op


In [ ]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# IMPORTACIÓN DE DATOS

## Fuentes de los datos:

>la DIAN proporciona información en sus estadisticas sobre Recaudo Bruto por Seccionales y Tipo de Impuesto 2005 - 2025, estos datos están en millones de $COP 

>El DANE en su Encuesta Anual Manufacturera recolecta datos de las empresas con el fin de obtener información básica del sector fabril que permita el conocimiento de su estructura, evolución y desarrollo. Utilizaremos esta base de datos para darnos acceso a la información sobre las empresas del sector industrial en particular del pago de impuestos.

## Acceso a datos:
>DIAN: https://www.dian.gov.co/dian/cifras/Paginas/EstadisticasRecaudo.aspx

>DANE: portal micro datos: https://microdatos.dane.gov.co/index.php/catalog/central/about

In [ ]:
# Mostrar el directorio actual 
directorio_inicial = os.getcwd()
print(f"Directorio inicial: {directorio_inicial}")

## Importación de dataset de la DIAN:
El objetivo es obtener datos para el intervalo 2018-2023 que estén completos y limpios con una nueva columna llamada departamento que será la que usemos para unir los datos del dane con la DIAN.

In [ ]:
#como queremos seguir en el directorio actual, pero los datos están cargados en otro folder, 
# vamos a generar una función que nos permita acceder a esa ruta donde estás los datasets y 
# ahorrar espacio el tiempo en el codigo, cosa que nos será util más adelante al importar 
# más datos, porque ya tendremos la ruta al folder definida.

# Ruta base de la carpeta "datasets"
ruta_datasets = r"C:\Users\Kayak\OneDrive\Escritorio\U\2025-2\IA_aplicada\proyecto\datasets" 
# Nombre del primer archivo que es tipo xlsx
Recaudo_Bruto = "Estadisticas-de-Recaudo-bruto-por-seccionales-y-tipo-de-impuesto-2005-2025.xlsx"

#creamos ruta para importar el priemer archivo
ruta_Recaudo_Bruto = os.path.join(ruta_datasets, Recaudo_Bruto)

# carga del archivo usando la ruta, saltandonos las primeras 5 filas y las últimas 10 porque son titulos o aclaraciones 
df_impuestos = pd.read_excel(ruta_Recaudo_Bruto,  skiprows=5,skipfooter=10)
#veamos que tipo de datos hay en cada columna y convirtamos a float los datos del recaudo que sean números.
# al ver el dataframe, notamos que la fila 1 y 2 están fusionadas por lo que debemos eliminar una dado que no aporta nada al analisis.
df_impuestos=df_impuestos.drop(index=0)

#de otro lado la columna AÑO debe ser rellenada, porque tiene muchos valores NaN debido a tener celdas combinadas en la base de datos.
# para ello usamos el método fillna con el parámetro method='ffill' (forward fill) que rellena los valores NaN con el último valor no nulo encontrado hacia adelante.
df_impuestos['AÑO'] = df_impuestos['AÑO'].ffill()

#Ahora bien, vamos a manejar datos solamente de 2018 a 2023, por lo que filtramos el dataframe para quedarnos solo con esos años.
df_impuestos['AÑO'] = pd.to_numeric(df_impuestos['AÑO'], errors='coerce')  # Convertir a numérico, forzando errores a NaN
df_impuestos = df_impuestos[(df_impuestos['AÑO'] >= 2018) & (df_impuestos['AÑO'] <= 2023)]
df_impuestos['AÑO'] = df_impuestos['AÑO'].astype(int)
df_impuestos.reset_index(inplace=True, drop=True) 

# Normalizar los nombres en la columna DIRECCIÓN_SECCIONAL
df_impuestos["DIRECCIÓN SECCIONAL"] = (
    df_impuestos["DIRECCIÓN SECCIONAL"]
    .str.strip()  # elimina espacios antes/después
    .str.upper() ) # pasa todo a mayúsculas

# los datos de la DIAN están en millones $COP, los del Dane están en miles de $COP
# por lo tanto, para que las unidades sean consistentes, convertiremos los datos de la DIAN a miles de $COP
#hay columnas tipo texto asi que vamos a ignorarlas para trabajar sin errores en una conversión de formato.
columnas_excluidas = ['AÑO', 'DIRECCIÓN SECCIONAL']
columnas_a_convertir = df_impuestos.columns.drop(columnas_excluidas)

# conversión solo a esas columnas
#dado que hay valores que no son número sino guiones, 
#usaremos .apply(pd.to_numeric, errors='coerce') para reemplazar automáticamente con NaN (Not a Number) 
#esos valores con guión, permitiendo que el resto de la conversión continúe.

df_impuestos[columnas_a_convertir] = df_impuestos[columnas_a_convertir].apply(pd.to_numeric, errors='coerce')
df_impuestos[columnas_a_convertir] = df_impuestos[columnas_a_convertir] * 1000
#convertimos los valores NaN a 0  
df_impuestos = df_impuestos.fillna(0)


In [ ]:

# Diccionario ciudad -> departamento (ajustado a mayúsculas)
ciudades_departamentos = {
    "ADUANAS BARRANQUILLA": "ATLÁNTICO",
    "ADUANAS BOGOTA": "CUNDINAMARCA",
    "ADUANAS CALI": "VALLE DEL CAUCA",
    "ADUANAS CARTAGENA": "BOLÍVAR",
    "ADUANAS DE CUCUTA": "NORTE DE SANTANDER",
    "ADUANAS DE MEDELLIN": "ANTIOQUIA",
    "IMPUESTOS BOGOTA": "CUNDINAMARCA",
    "GRANDES CONTRIBUYENTES BOGOTA": "BOGOTÁ, D.C.",
    "OPERERATIVA DE GRANDES CONTRIBUYENTES": "BOGOTÁ, D.C.",
    "OPERATIVA GRANDES CONTRIBUYENTES": "BOGOTÁ, D.C.",

    "ARAUCA": "ARAUCA",
    "ARMENIA": "QUINDÍO",
    "BARRANCABERMEJA": "SANTANDER",
    "BARRANQUILLA": "ATLÁNTICO",
    "BUCARAMANGA": "SANTANDER",
    "BUENAVENTURA": "VALLE DEL CAUCA",
    "CALI": "VALLE DEL CAUCA",
    "CARTAGENA": "BOLÍVAR",
    "CUCUTA": "NORTE DE SANTANDER",
    "FLORENCIA": "CAQUETÁ",
    "GIRARDOT": "CUNDINAMARCA",
    "IBAGUE": "TOLIMA",
    "INIRIDA": "GUAINÍA",
    "IPIALES": "NARIÑO",
    "LETICIA": "AMAZONAS",
    "MAICAO": "LA GUAJIRA",
    "MANIZALES": "CALDAS",
    "MEDELLIN": "ANTIOQUIA",
    "MITU": "VAUPÉS",
    "MONTERIA": "CÓRDOBA",
    "NEIVA": "HUILA",
    "PALMIRA": "VALLE DEL CAUCA",
    "PASTO": "NARIÑO",
    "PEREIRA": "RISARALDA",
    "POPAYAN": "CAUCA",
    "PUERTO ASIS": "PUTUMAYO",
    "PUERTO CARREÑO": "VICHADA",
    "QUIBDO": "CHOCÓ",
    "RIOHACHA": "LA GUAJIRA",
    "SAN ANDRES": "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA",
    "SAN JOSE DEL GUAVIARE": "GUAVIARE",
    "SANTA MARTA": "MAGDALENA",
    "SINCELEJO": "SUCRE",
    "SOGAMOSO": "BOYACÁ",
    "TULUA": "VALLE DEL CAUCA",
    "TUNJA": "BOYACÁ",
    "URABA": "ANTIOQUIA",
    "VALLEDUPAR": "CESAR",
    "VILLAVICENCIO": "META",
    "YOPAL": "CASANARE"
}

# Diccionario departamento -> código
departamento_codigos = {
    "ANTIOQUIA": 5,
    "ATLÁNTICO": 8,
    "BOGOTÁ, D.C.": 11,
    "BOLÍVAR": 13,
    "BOYACÁ": 15,
    "CALDAS": 17,
    "CAQUETÁ": 18,
    "CAUCA": 19,
    "CESAR": 20,
    "CÓRDOBA": 23,
    "CUNDINAMARCA": 25,
    "CHOCÓ": 27,
    "HUILA": 41,
    "LA GUAJIRA": 44,
    "MAGDALENA": 47,
    "META": 50,
    "NARIÑO": 52,
    "NORTE DE SANTANDER": 54,
    "QUINDÍO": 63,
    "RISARALDA": 66,
    "SANTANDER": 68,
    "SUCRE": 70,
    "TOLIMA": 73,
    "VALLE DEL CAUCA": 76,
    "ARAUCA": 81,
    "CASANARE": 85,
    "PUTUMAYO": 86,
    "ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA": 88,
    "AMAZONAS": 91,
    "GUAINÍA": 94,
    "GUAVIARE": 95,
    "VAUPÉS": 97,
    "VICHADA": 99
}

#eliminación de filas los totales por año para poder unir las bases y generar columna dpto.
df_impuestos_arreglado = df_impuestos[~df_impuestos["DIRECCIÓN SECCIONAL"].str.contains("TOTAL AÑO", case=False, na=False)].copy()
# Asignación de departamento
df_impuestos_arreglado["departamento"] = df_impuestos_arreglado["DIRECCIÓN SECCIONAL"].map(ciudades_departamentos)

# 4. Asignación de  código de departamento
df_impuestos_arreglado["dpto"] = df_impuestos_arreglado["departamento"].map(departamento_codigos)
df_impuestos_arreglado['dpto'] = df_impuestos_arreglado['dpto'].apply(pd.to_numeric, errors='coerce')
df_impuestos_arreglado = df_impuestos_arreglado.dropna(subset=["dpto"])
df_impuestos_arreglado["dpto"] = df_impuestos_arreglado["dpto"].astype(int)

In [ ]:
df_impuestos_arreglado

In [ ]:
df_impuestos_arreglado.info(   )

## Importación de dataset del DANE: Encuesta Anual Manufacturera

In [ ]:
#indicación de la ruta de los datasets y el separador de los archivos csv
ruta_datasets = r'C:\Users\Kayak\OneDrive\Escritorio\U\2025-2\IA_aplicada\proyecto\datasets'
CSV_SEPARATOR = ';' 

# Diccionario de configuración de archivos y formato exacto por año
archivos = {
    2018: {'nombre': 'EAM_ANONIMIZADA_2018.csv', 'formato': 'csv'},
    2019: {'nombre': 'EAM_ANONIMIZADA_2019.csv', 'formato': 'csv'},
    2020: {'nombre': 'EAM_ANONIMIZADA_2020.xlsx', 'formato': 'xlsx'}, 
    2021: {'nombre': 'EAM_ANONIMIZADA_2021.xlsx', 'formato': 'xlsx'}, 
    2022: {'nombre': 'EAM_ANONIMIZADA_2022.xlsx', 'formato': 'xlsx'},
    2023: {'nombre': 'EAM_ANONIMIZADA_2023.xlsx', 'formato': 'xlsx'}
}

# Diccionario de decodificación y selección de columnas
columnas_renombre = {
    'dpto': 'DEPARTAMENTO',
    'periodo': 'AÑO',
    'C3R20C3': 'Impuesto de Renta para la Equidad',
    'c3r25c3': 'Impuesto del 4 x 1 000',
    'c3r37c3': 'Impuesto de Industria y Comercio',
    'c3r38c3': 'Impuesto Predial y sobre Vehículos'
}
columnas_seleccion = list(columnas_renombre.keys()) # Las columnas originales que buscamos
columnas_impuestos = list(columnas_renombre.values())[2:] # Solo los nombres de los impuestos finales

# Lista para almacenar los DataFrames procesados anualmente
list_df_anuales = []

# --- 2. Bucle de Procesamiento ---
for año, dataset in archivos.items():
    nombre_archivo = dataset['nombre']
    formato_archivo = dataset['formato']
    ruta_archivo = os.path.join(ruta_datasets, nombre_archivo)
    df_EAM = None

    # A. Importación Condicional
    try:
        if formato_archivo == 'xlsx':
            df_EAM = pd.read_excel(ruta_archivo) 
        elif formato_archivo == 'csv':
            df_EAM = pd.read_csv(ruta_archivo, sep=CSV_SEPARATOR)
        else:
            print(f" Error: Formato '{formato_archivo}' no reconocido.")
            continue
        
        # Estandarización de nombres: convierte los nombres de columnas a minúsculas
        # Esto soluciona la mayoría de los KeyErrors si el archivo no respeta mayúsculas/minúsculas.
        df_EAM.columns = df_EAM.columns.str.lower().str.strip()
        print(f"✅ Importado y nombres estandarizados: {nombre_archivo}")
        
    except FileNotFoundError:
        print(f" Error: Archivo NO encontrado en la ruta para el año {año}.")
        continue
    except Exception as e:
        print(f" Error al leer el archivo {nombre_archivo}. Detalles: {e}")
        continue

    # B. Selección y Renombramiento (Idéntico a tu referencia, pero con nombres limpios)
    # 1. Aseguramos que la selección se haga con los nombres limpios (minúsculas)
    columnas_a_seleccionar_limpias = [col.lower() for col in columnas_seleccion]
    
    try:
        # Aquí se aplica la selección de columnas.
        df_EAM_util = df_EAM[columnas_a_seleccionar_limpias].copy()
    except KeyError as e:
        print(f" ERROR CRÍTICO en el año {año}: Columnas no encontradas. Faltan: {e}. Revisar el archivo.")
        print(f"Columnas disponibles: {df_EAM.columns.tolist()}")
        continue

    # 2. Renombramos usando los nombres limpios como clave
    columnas_renombre_limpio = {k.lower(): v for k, v in columnas_renombre.items()}
    df_EAM_util = df_EAM_util.rename(columns=columnas_renombre_limpio)
    

    # C. Conversión de Tipos de Datos (Necesario para sumar después de leer CSV/TXT)
    for col in columnas_impuestos:
        df_EAM_util[col] = pd.to_numeric(df_EAM_util[col], errors='coerce')

    # D. Agrupación y Suma por Departamento (Igual que en tu referencia)
    df_EAM_sumado_por_dpto = df_EAM_util.groupby('DEPARTAMENTO').sum(numeric_only=True).reset_index()

    # E. Corrección del Campo 'AÑO'
    df_EAM_sumado_por_dpto['AÑO'] = año

    # F. Almacenamiento para concatenación
    list_df_anuales.append(df_EAM_sumado_por_dpto)


# --- 3. Consolidación Final ---
# ¡Concatenación vertical (apilamiento) de todos los resúmenes anuales!
df_EAM_consolidado = pd.concat(list_df_anuales, ignore_index=True)

print("\n--- PROCESO TERMINADO ---")
print(f"DataFrames consolidados para los años: {min(archivos.keys())} a {max(archivos.keys())}")
print(f"Total de registros consolidados (Filas): {len(df_EAM_consolidado)}")
print("\nPrimeras filas del DataFrame Consolidado:")
print(df_EAM_consolidado.head(10))

In [ ]:
df_EAM_consolidado.rename(columns={'DEPARTAMENTO': 'dpto'}, inplace=True)

In [ ]:
df_EAM_consolidado

In [ ]:
df_EAM_consolidado

In [ ]:

df_EAM_impuestos = pd.merge(df_EAM_consolidado, df_impuestos_arreglado, on='dpto', how='outer')

In [ ]:
df_EAM_impuestos.head(20)